In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import geopandas as gpd

import extracting_points_from_polygons as pp

In [1]:
# aois = ['campus_lagoon','carpinteria','gaviota','point_conception']
aoi = 'gaviota'

# years can be any subset from [2012, 2014, 2016, 2018, 2020]
# except if aoi = point conception (no pts for 2016)
years = [2016]

In [ ]:
all_pts = []

for year in years:
    
    # open polygons
    fp = pp.path_to_polygons(aoi,year)
    polys = gpd.read_file(fp)
    
    polys_ice = polys.loc[polys.iceplant==1]
    polys_ice.reset_index(inplace=True, drop=True)
    pts_ice = pp.naip_sample_proportion_no_warnings(polys_ice, 
                                                    polys.naip_id[0], 
                                                    0.9)  # sample 90% of pts in polygon

    print('*** '+str(year)+' # ice plant pts sampled')
    print(pts_ice.shape[0])
    pts_ice.polygon_id.value_counts()

    
    polys_nonice = polys.loc[polys.iceplant==0]
    polys_nonice.reset_index(inplace=True, drop=True)
    pts_nonice = pp.naip_sample_sliding_no_warnings(polys_nonice, polys.naip_id[0], 
                                                    0.9, # sample 90% of pts in polygon, but at most 1500
                                                    1500)

    print('*** '+str(year)+' # non ice plant pts sampled')
    print(pts_nonice.shape[0])
    pts_nonice.polygon_id.value_counts()
    
    pts = pd.concat([pts_ice,pts_nonice])
    print('*** '+str(year)+' # proportions')
    pp.iceplant_proportions(pts.iceplant)
    
    all_pts.append(pts)
    
    print( '---------------------------------------')

# Save points

In [ ]:
for i in range(0,len(years)):
    fp = pp.path_points_csv(aoi,years[i])
    pts[i].to_csv(fp)
        
